# Project 3D Modeling and Image Based Rendering

## 0 Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# General imports
import cv2
import numpy as np
from glob import glob
import os.path
from os import makedirs

BASE_DATA_PATH = "/Users/kobe/Documents/School/2024-2025/3D_Modeling_and_Image_Based_Rendering/project/dataset"
CURRENT_DATASET = BASE_DATA_PATH + "/GrayCodes_HighRes"
CHESS_PATH = CURRENT_DATASET + "/chess"
RAW_PATH = CURRENT_DATASET + "/raw"
UNDIST_PATH = CURRENT_DATASET + "/undistorted"
PATTERN_PATH = CURRENT_DATASET + "/patterns"

if not os.path.exists(UNDIST_PATH):
	makedirs(UNDIST_PATH)

if not os.path.exists(PATTERN_PATH):
	makedirs(PATTERN_PATH)

In [ ]:
def getFilesSortedNumeric(glob_pattern: str):
	files = glob(glob_pattern)
	return sorted(files, key=lambda f: int(os.path.splitext(os.path.basename(f))[0]))

## 1 Camerakalibratie

### Kalibreer camera

In [ ]:
from calibration import combine_extrinsic_vecs, intrinsic_calibration

chess_files = glob(f"{CHESS_PATH}/*.jpg")
error, intrinsic, distortion, rotation_matrices, translation_vectors, image_size = intrinsic_calibration(chess_files, (7,9))
extrinsics = combine_extrinsic_vecs(rotation_matrices, translation_vectors)

np.savez(f"{CURRENT_DATASET}/camCalibration.npz", intrinsic=intrinsic, distortion=distortion, extrinsics=extrinsics, img_size=image_size)

### Load kalibration data

In [ ]:
calibration = np.load(f"{CURRENT_DATASET}/camCalibration.npz")
intrinsic = calibration["intrinsic"]
distortion = calibration["distortion"]
extrinsics = calibration["extrinsics"]
height, width = calibration['img_size']

### Undistort alle afbeelding

In [ ]:
from undistort import undistortAllViews

undistortAllViews(RAW_PATH, UNDIST_PATH, intrinsic, distortion)

### Visualize poses

In [ ]:
from visualizeCameraPoints import drawCameraPoints

drawCameraPoints(height, width, intrinsic, extrinsics)

## 2 Structured Light

### Genereer patronen

In [ ]:
from GrayCodeEncoder import GrayCodeEncoder


ROWS = 1080
COLS = 1920
DEPTH = 10

encoder = GrayCodeEncoder(ROWS, COLS, DEPTH)
for depth, pattern in enumerate(encoder.patterns):
	output_filename = f"{PATTERN_PATH}/{depth}.jpg"
	cv2.imwrite(output_filename, pattern)
	print(f"\rSaved {os.path.basename(output_filename)}", end='')


### Decode graycode images

In [ ]:
from GrayCodeDecoder import GrayCodeDecoder

THRESHOLD = 8
for view in os.listdir(str(UNDIST_PATH)):
	print(f"Decoding {view}...")
	decoder = GrayCodeDecoder([cv2.imread(file, cv2.IMREAD_GRAYSCALE) for file in getFilesSortedNumeric(f"{UNDIST_PATH}/{view}/[0-9][0-9].jpg")])
	codes, mask = decoder.decode(THRESHOLD)
	np.savez_compressed(f"{CURRENT_DATASET}/{view}_decoded.npz", codes=codes, mask=mask, threshold=THRESHOLD)

### Load decoded data

In [ ]:
def createViewDict(view: str):
	data = np.load(f"{CURRENT_DATASET}/{view}_decoded.npz")
	return {
		"codes": data["codes"],
		"mask": data["mask"],
		"threshold": data['threshold']
	}

decoded_views = {
	 f"{view}": createViewDict(view) for view in os.listdir(UNDIST_PATH)
}

### Find matches between views

In [ ]:
from matcher import correspond

kp0, kp1, dMatches, matches = correspond(decoded_views)
np.savez_compressed(f"{CURRENT_DATASET}/matches.npz", matches=matches, allow_pickle=True)

### Load matches

In [ ]:
loaded_data = np.load(f"{CURRENT_DATASET}/matches.npz", allow_pickle=True)
matches = loaded_data['matches'].item()
loaded_data.close()

### Draw Matches

In [ ]:
from matcher import drawMatches

img0 = cv2.imread(f"{UNDIST_PATH}/view0/00.jpg")
img1 = cv2.imread(f"{UNDIST_PATH}/view1/00.jpg")
drawMatches(matches, img0, img1)

## 3 Reconstructie Puntenwolk

### Essentiele matrix berekenen

In [ ]:
def getKeyPoints(matches):
	kp0 = []
	kp1 = []
	for match in matches.values():
		kp0.append(match[0])
		kp1.append(match[1])
	return np.array(kp0), np.array(kp1)

In [ ]:
from essentialMatrixGeneration import generateEssentialMatrix

kp0, kp1 = getKeyPoints(matches)
essential_matrix, mask = generateEssentialMatrix(kp0, kp1, intrinsic)

In [ ]:
np.savez(f"{CURRENT_DATASET}/essentialMatrix.npz", essential=essential_matrix)

### Recover Pose of cameras

In [ ]:
from poseRecovery import recoverPose
from calibration import combine_extrinsic_vec

kp0, kp1 = getKeyPoints(matches)
rotation_cam1, translation_cam1, mask_pose = recoverPose(essential_matrix, kp0, kp1, intrinsic, mask)
np.savez(f"{CURRENT_DATASET}/extrinsics.npz", view0=combine_extrinsic_vec(np.eye(3), np.zeros((3,1))), view1=combine_extrinsic_vec(rotation_cam1, translation_cam1))

### Visualize poses of the cameras

In [ ]:
from calibration import combine_extrinsic_vec
from visualizeCameraPoints import drawCameraPoints

cam0_extrinsic = combine_extrinsic_vec(np.eye(3), np.zeros((3,1)))
cam1_extrinsic = combine_extrinsic_vec(rotation_cam1, translation_cam1)
drawCameraPoints(height, width, intrinsic, [cam0_extrinsic, cam1_extrinsic])

### Triangulate points

In [ ]:
from triangulation import triangulatePointsCustom

kp0, kp1 = getKeyPoints(matches)
proj0 = intrinsic @ np.hstack((np.eye(3), np.zeros((3,1))))
proj1 = intrinsic @ np.hstack((rotation_cam1, translation_cam1))
points_4d_hom = triangulatePointsCustom(proj0, proj1, kp0, kp1)
np.savez(f"{CURRENT_DATASET}/pointCloud.npz", points=points_4d_hom)

### Visualize triangulated points

In [ ]:
import open3d
import open3d.visualization
import open3d.cpu.pybind.utility as utility

from visualizeCameraPoints import createLineSet


points3D = points_4d_hom[:3, :] / points_4d_hom[3, :]

points = utility.Vector3dVector(points3D.T)
point_cloud = open3d.geometry.PointCloud(points)
open3d.visualization.draw_geometries([point_cloud, createLineSet(height, width, intrinsic, cam0_extrinsic), createLineSet(height, width, intrinsic, cam1_extrinsic)])

## 4 Plane-Sweep

### Load data needed for this step

In [5]:
# Calibration
calibration_data = np.load(f"{CURRENT_DATASET}/camCalibration.npz")
height, width = calibration_data['img_size']
intrinsic = calibration_data['intrinsic']
point_cloud = np.load(f"{CURRENT_DATASET}/pointCloud.npz")['points']

extrinsics = np.load(f"{CURRENT_DATASET}/extrinsics.npz")
cam0_extrinsic = extrinsics['view0']
cam0_extrinsic = np.vstack((cam0_extrinsic, [0,0,0,1]))
cam1_extrinsic = extrinsics['view1']
cam1_extrinsic = np.vstack((cam1_extrinsic, [0,0,0,1]))

### Get depth map

In [ ]:
from planeSweep import getDepthMap, interpolateBetweenCams


for depth in range(11):
	depth_map = getDepthMap(intrinsic, interpolateBetweenCams(cam0_extrinsic, cam1_extrinsic, depth/10), point_cloud, (height, width))
	cv2.imwrite(f"{CURRENT_DATASET}/depth_{depth/10}.jpg", depth_map)

### Center of point cloud

In [ ]:
from planeSweep import interpolateBetweenCams
virtual_extrinsic = interpolateBetweenCams(cam0_extrinsic, cam1_extrinsic, 0.5)
virtual_extrinsic = np.vstack((virtual_extrinsic, [0,0,0,1]))

pc_center = (point_cloud[:3, :] / point_cloud[3, :]).mean(axis=1)

In [60]:
corners = np.array([
	[0,0, 1],
	[4752,0, 1],
	[4752, 3168, 1],
	[0,3168, 1],
], dtype=np.float64).T

def screenToWorld(intrinsic, extrinsic, distance, points):
	"""Converts a screen space coordinate to a world space coordinate for a given distance

	Args:
		intrinsic (3x3 matrix): The intrinsic matrix of the camera
		extrinsic (4x4 matrix): The extrinsic matrix of the camera
		distance (float): The distance from the camera the point should be
		points (3xN matrix): The points to transform to worldSpace

	Returns:
		4xN matrix:  The points transformed to worldspace
	"""
	intrinsic_inv = np.linalg.inv(intrinsic)
	Pc =  intrinsic_inv @ points * np.array([distance]*4).reshape((1, 4))
	Pc_hom = np.vstack((Pc, np.ones((1,4), dtype=np.float64)))
	Pw_hom = np.linalg.inv(extrinsic) @ Pc_hom
	return Pw_hom

def worldToScreen(intrinsic, extrinsic, points):
	"""Transforms worldspace coordinates to screen space

	Args:
		intrinsic (3x3 Matrix): The intrinsic matrix of the camera
		extrinsic (4x4 Matrix): The extrinsic matrix of the camera
		points (4xN): The points to transform

	Returns:
		2xN Matrix: The transformed points
	"""
	cam = extrinsic @ points
	screen = intrinsic @ cam[:3, :]
	return screen[:2, :] / screen[2, :]

def getDepthRangeFromPointCloud(point_cloud, extrinsic, bins):
	points = extrinsic @ point_cloud
	points = points[:-1, :] / points[-1, :]
	z_values = np.unique(np.round(points[-1, :], 5))
	bin_width = len(z_values) // bins
	residual = len(z_values) % bin_width
	reshaped = z_values[:-residual].reshape((-1, bin_width))
	binned = reshaped.mean(axis=1)
	return np.hstack((points[-1, 0]-1e-2, binned, z_values[-residual:].mean(), points[-1, -1]+1e-2)) 

In [61]:
def drawRectOnImg(img, points):
	outimg = img
	for i in range(4):
		j = (i + 1) % 4
		outimg = cv2.line(outimg, points[:, i].astype(np.int32), points[:, j].astype(np.int32), (255, 255, 255))
	return outimg
		


interpolated_image = np.zeros((height, width))
interpolated_image = interpolated_image.astype(np.float64)

dist = np.linalg.norm(pc_center - virtual_extrinsic[:-1, -1])

global_diff = np.ones_like(interpolated_image) * 255

depths = getDepthRangeFromPointCloud(point_cloud, virtual_extrinsic, 100)
print(depths.size)
cam0_image = cv2.imread(f"{UNDIST_PATH}/view0/00.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float64)
cam1_image = cv2.imread(f"{UNDIST_PATH}/view1/00.jpg", cv2.IMREAD_GRAYSCALE).astype(np.float64)

blurr_kernel = np.array([1/25] * 25).reshape((5,5))



for i, depth in enumerate(depths):
	print(f"Depthplane: {depth:.3f}, {i/len(depths) * 100:.2f}%", end='\r')
	# Deproject the virtual image plane to world coordinates
	world_corners = screenToWorld(intrinsic, virtual_extrinsic, depth, corners)
	cam0_screen = worldToScreen(intrinsic, cam0_extrinsic, world_corners)
	cam1_screen = worldToScreen(intrinsic, cam1_extrinsic, world_corners)

	cam0_homography, _ = cv2.findHomography(cam0_screen.T, corners[:-1, :].T)
	cam1_homography, _ = cv2.findHomography(cam1_screen.T, corners[:-1, :].T)


	cam0_image_persp = cv2.warpPerspective(cam0_image, cam0_homography, (4752, 3168))
	cam0_image_persp_blur = cv2.filter2D(cam0_image_persp, -1, blurr_kernel)
	cam1_image_persp = cv2.warpPerspective(cam1_image, cam1_homography, (4752, 3168))
	cam1_image_persp_blur = cv2.filter2D(cam1_image_persp, -1, blurr_kernel)

	diff = cv2.absdiff(cam0_image_persp, cam1_image_persp)
	indices = (global_diff > diff) # & (diff < 50)
	global_diff[indices] = diff[indices]
	interpolated_image[indices] = (cam0_image_persp[indices] + cam1_image_persp[indices]) / 2

print()
cv2.imwrite(f"{CURRENT_DATASET}/interpolated_plane_depth.jpg", interpolated_image.astype(np.uint8))


103
Depthplane: 2.223, 99.03%


True

In [ ]:
geometries = [point_cloud]

sphere = open3d.geometry.TriangleMesh.create_sphere(0.1, 20)
sphere.translate(pc_center)
geometries.append(sphere)
cam0_lineSet = createLineSet(height, width, intrinsic, cam0_extrinsic)
geometries.append(cam0_lineSet)
cam1_lineSet = createLineSet(height, width, intrinsic, cam1_extrinsic)
geometries.append(cam1_lineSet)
edges = lines = np.array([
        [0, 1],  # Edge 1
        [1, 2],  # Edge 2
        [2, 3],  # Edge 3
        [3, 0]   # Edge 4 (closes the rectangle)
    ])
# projected_area = open3d.geometry.LineSet(utility.Vector3dVector(Pw.T), utility.Vector2iVector(edges))
# geometries.append(projected_area)
# for i in range(1, 10):
#     geometries.append(createLineSet(height, width, intrinsic, interpolateBetweenCams(cam0_extrinsic, cam1_extrinsic, i / 10)))
geometries.append(createLineSet(height, width, intrinsic, virtual_extrinsic))
    
open3d.visualization.draw_geometries(geometries)

## 5 Eigen Dataset Capteren

## 6 Reconstrucite Camera-Camera-Projector

## 7 Light Field Displays